In [1]:
import sqlite3
import pandas as pd
import numpy as np

# UNWTO Data Update
- **Last Updated : 24.04.2024**

## UNWTO Data Insert

### UN Data Cleaning

#### Define Function for UN Data Obtaining

In [107]:
def get_un_data(un_path, sheet_num, skip_rows, df_csv_un_africa, indicator_list_un, year_range, sheet_dict):
    # load in sheet 'Inbound Tourism-Arrivals' from 'unwto-all-data-download_21'
    df=pd.read_excel(un_path,sheet_name=sheet_num,skiprows=skip_rows)
    df_wto_africa = df[df['C.'].isin(df_csv_un_africa['country_code'])]

    df_wto_africa.loc[:, 'Unnamed: 5'].fillna(df_wto_africa['Unnamed: 6'], inplace=True)
    df_wto_africa.loc[:, 'Unnamed: 5'].fillna(df_wto_africa['Unnamed: 7'], inplace=True)
    df1= df_wto_africa.query(f"`S.` in {indicator_list_un}") #filter on total arrivals and arrivals variables (s column is for indicator)
    df1.rename(columns={'Unnamed: 5': 'indicator'}, inplace=True) # rename to indicator column
    c = list(year_range)#list years
    country_dict = dict(zip(df_csv_un_africa['country_code'], df_csv_un_africa['country'])) #create a map from country code to country
    df2 = pd.melt(df1, id_vars=['C.','S.', 'indicator', 'Units'], value_vars=c) # collapse un df
    df2['country'] = df2['C.'].map(country_dict) # add country column
    df2.value.replace('..',np.NaN,inplace=True) #replace with NaN values
    df2 = df2.merge(df_csv_un_africa[['country_code', 'iso3_code']], left_on='C.',right_on='country_code', how='left')
    df2.drop(columns=['country_code'], inplace=True, axis = 1)
    df2.rename(columns={'variable' : 'year'}, inplace=True) # rename variable column
    df2['indicator_source_cat'] = sheet_dict[sheet_num]
    df2.dropna(subset=['indicator'], inplace=True, axis = 0)

    return df2

#### Define UN Indicator Groups

In [108]:
xlsx = pd.ExcelFile(r'../../datasets/unwto/unwto-all-data-download_112023.xlsx')

# Get the sheet names as a list
sheet_names = xlsx.sheet_names

# Create a dictionary mapping sheet numbers to names
sheet_dict = {i: sheet_names[i] for i in range(len(sheet_names))}

print(sheet_dict)

{0: 'Index', 1: ' Inbound Tourism-Arrivals', 2: 'Inbound Tourism-Regions', 3: 'Inbound Tourism-Purpose', 4: 'Inbound Tourism-Transport', 5: 'Inbound Tourism-Accommodation', 6: 'Inbound Tourism-Expenditure', 7: 'Domestic Tourism-Trips', 8: 'Domestic Tourism-Accommodation', 9: 'Outbound Tourism-Departures', 10: 'Outbound Tourism-Expenditure', 11: 'Tourism Industries', 12: 'Employment'}


#### Define UN INdicator IDs

In [109]:
dict_un_indicator = {
    1 : [1.1, 1.2, 1.3], 
    2 : [1.6,1.7,1.8,1.9,1.10,1.11,1.12,1.13], 
    3 : [1.14,1.15,1.18], 
    4: [1.19,1.20,1.21,1.22], 
    6 : [1.33,1.34,1.35], 
    7 : [2.1,2.2,2.3], 
    9 : [3.1,3.2,3.3],
    10:[3.4,3.5,3.6],
    11: [4.13,4.14,4.15,4.16,4.17,4.18],
    12: [5.1,5.2,5.3,5.4,5.5,5.6,5.7]
}

### Obtain UN Complete Data from Excel

In [110]:
df_csv_un_africa = pd.read_excel(r'../../datasets/unwto/countryContUNWTO.xlsx')
df_list = {}
for i in dict_un_indicator.keys():
    df = get_un_data(r'../../datasets/unwto/unwto-all-data-download_112023.xlsx', i, 2, df_csv_un_africa, dict_un_indicator[i], range(1995, 2023), sheet_dict)
    df_list[i] = df

/var/folders/6c/vby383sn3xlg0j_c253f0sf40000gn/T/ipykernel_31442/1793109855.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wto_africa.loc[:, 'Unnamed: 5'].fillna(df_wto_africa['Unnamed: 6'], inplace=True)
/var/folders/6c/vby383sn3xlg0j_c253f0sf40000gn/T/ipykernel_31442/1793109855.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wto_africa.loc[:, 'Unnamed: 5'].fillna(df_wto_africa['Unnamed: 7'], inplace=True)
/var/folders/6c/vby383sn3xlg0j_c253f0sf40000gn/T/ipykernel_31442/1793109855.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

### Add Relevant Columns for Final Insertion Into Table

In [111]:
for i in df_list:
    df_list[i]['indicator_ddt_cat'] =  df_list[i]['indicator_source_cat']
    df_list[i]['indicator_source'] =  'UNWTO'
    df_list[i]['indicator_source_name'] =  df_list[i]['indicator']
    df_list[i]['indicator_year'] =  df_list[i]['year']
    df_list[i]['iso3_country_code'] =  df_list[i]['iso3_code']
    df_list[i]['indicator_value'] =  df_list[i]['value']
    df_list[i]['units'] =  df_list[i]['Units']


### Define DDT Indicator Names for Each UN Indicator

In [112]:
df_ddt_indic_name_un = {
    'Total arrivals' : 'Total International Arrivals',
    'Overnights visitors (tourists)' : 'Total International Overnight Visitors (Tourists)',
    'Same-day visitors (excursionists)' : 'Total International Same-day Visitors (Excursionists)',
    'Personal' : 'Total Personal Travelers',
    'Business and professional' : 'Total Business and Professional Travelers',
    'Air' : 'Total Travelers by Air',
    'Water' : 'Total Travelers by Water',
    'Land' : 'Total Travelers by Land',
    'Travel' : 'Expenditure on Travel',
    'Passenger transport': 'Expenditure on Passenger Transport',
    'Total trips' : 'Total Domestic Trips',
    'Total departures' : 'Total International Departures',
    'Number of establishments' : 'Total Number of Establishments',
    'Number of rooms' : 'Total Number of Rooms',
    'Number of bed-places' : 'Total Number of Bed Places', 
    'Occupancy rate / rooms': 'Total Occupancy Rate / Rooms' ,
    'Occupancy rate / bed-places' : 'Total Occupancy Rate / Bed-Places', 
    'Average length of stay' : 'Average Length of Stay',
    'Accommodation services for visitors (hotels and similar establishments)' : 'Total Employment in Accommodation Services for Visitors (Hotels and Similar Establishments)',
    'Other accommodation services' : 'Total Employment in Other Accommodation Services',
    'Food and beverage serving activities' : 'Total Employment in Food and Beverage Serving Activities', 
    'Passenger transportation' : 'Total Employment in Passenger Transportation',
    'Travel agencies and other reservation services activities': 'Total Employment in Travel agencies and Other Reservation Services Activities',
    'Other tourism industries' : 'Total Employment in Other Tourism Industries',
    'Total' : 'Total'
}
for each in df_list[2]['indicator'].unique():
    df_ddt_indic_name_un[each] = 'Total Travelers Originating From ' + each

### Add `indicator_ddt_name` for each un_df

In [113]:
for each in df_list:
    df_list[each]['indicator_ddt_name'] = df_list[each]['indicator'].apply(lambda x: df_ddt_indic_name_un[x])

#### Change Values of Total

In [114]:
for each in df_list:
    each_df = df_list[each]
    if 'Total' in each_df.indicator_ddt_name.unique():
        new_name = 'Total (' + each_df['indicator_ddt_cat'].unique()[0] +')'
        # Update only 'indicator_ddt_name' column where its value is 'Total'
        each_df.loc[each_df['indicator_ddt_name'] == 'Total', 'indicator_ddt_name'] = new_name
    else:
        continue

## Adding Units Column to `indicator_table`
- !No Need To Run

In [115]:
# Connect to the database (it will be created if it doesn't exist)
conn = sqlite3.connect('../../datasets/data_discovery_db.sqlite')
cursor = conn.cursor()

# Step 1: Add a new column 'units' to the indicator_table
cursor.execute("ALTER TABLE indicator_table ADD units TEXT;")

# Step 2: Update the 'units' column based on the conditions

# Update rows where indicator_ddt_name is 'Total International Arrivals' or 'Total International Departures'
cursor.execute("""
    UPDATE indicator_table
    SET units = 'Total'
    WHERE indicator_ddt_name IN ('Total International Arrivals', 'Total International Departures');
""")

# Update rows for other values in indicator_ddt_name
cursor.execute("""
    UPDATE indicator_table
    SET units = 'Current USD'
    WHERE units IS NULL;
""")

# Commit the changes and close the connection
conn.commit()
conn.close()


## Create Backup of `indicator_table`

In [116]:

# Connect to the SQLite3 database
conn = sqlite3.connect('../../datasets/data_discovery_db.sqlite')
cursor = conn.cursor()

# Create a backup table with the same structure as the original
cursor.execute("CREATE TABLE indicator_table_backup AS SELECT * FROM indicator_table WHERE 1=0;")

# Copy the data from the original table to the backup table
cursor.execute("INSERT INTO indicator_table_backup SELECT * FROM indicator_table;")

# Commit the changes and close the connection
conn.commit()
conn.close()


## Create Function For Inserting UN Indicators Into `indicator_table`

In [117]:
import uuid

def insert_un_indicator(df, conn):
    df_filtered = df[['S.', 'indicator', 'Units', 'indicator_source_cat', 
                      'indicator_ddt_cat', 'indicator_source', 'indicator_source_name', 
                      'units', 'indicator_ddt_name']]
    df_filtered = df_filtered.drop_duplicates()
    
    # Generate UUIDs for each row and add as a new column 'indicator_id'
    df_filtered['indicator_id'] = [str(uuid.uuid4()) for _ in range(len(df_filtered))]
    
    cursor = conn.cursor()
    
    # Construct the SQL statement for insertion with ON CONFLICT clause for SQLite
    sql = """
        INSERT INTO indicator_table (
            indicator_id, 
            indicator_source_id, 
            indicator_source_name, 
            indicator_ddt_name, 
            indicator_source, 
            indicator_source_cat, 
            indicator_ddt_cat,
            units
        ) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ON CONFLICT(indicator_id) DO UPDATE SET
            indicator_source_id = excluded.indicator_source_id,
            indicator_source_name = excluded.indicator_source_name,
            indicator_ddt_name = excluded.indicator_ddt_name,
            indicator_source = excluded.indicator_source,
            indicator_source_cat = excluded.indicator_source_cat,
            indicator_ddt_cat = excluded.indicator_ddt_cat,
            units = excluded.units;
    """
    
    # Use df.iterrows() to iterate over dataframe rows and execute SQL insert statement for each row
    for _, row in df_filtered.iterrows():
        try:
            cursor.execute(sql, (
                row['indicator_id'],
                row['S.'],
                row['indicator_source_name'],
                row['indicator_ddt_name'],
                row['indicator_source'],
                row['indicator_source_cat'],
                row['indicator_ddt_cat'],
                row['units']
            ))
        except Exception as e:
            print(f"Failed to insert/update due to error: {e}")
            # Optionally handle specific errors or rollback/ignore based on your requirements
    
    # Commit the changes
    conn.commit()



## Insert UNWTO Indicators into `indicator_table`

In [118]:
# Connect to the SQLite3 database
conn = sqlite3.connect('../../datasets/data_discovery_db.sqlite')

for i in df_list:
    df = df_list[i]
    insert_un_indicator(df, conn)

# Close the connection
conn.close()


## View Changes in `indicator_table`

In [119]:
# Connect to the database (it will be created if it doesn't exist)
conn = sqlite3.connect('../../datasets/data_discovery_db.sqlite')
cursor = conn.cursor()

# Retrieve data from each table and store in separate DataFrames
#country_df = pd.read_sql_query("SELECT * FROM country_table", conn)
#country_indicator_value_df = pd.read_sql_query("SELECT * FROM country_indicator_value_table", conn)
indicator_df = pd.read_sql_query("SELECT * FROM indicator_table", conn)

# Close the database connection
conn.close()


In [120]:
#indicator_df.loc[5]['indicator_source_id']
indicator_df.query('indicator_source_id == "1.1"')

,indicator_id,indicator_source_name,indicator_ddt_name,indicator_source,indicator_source_id,indicator_source_cat,indicator_ddt_cat,units
0,01fbcfcd-cf29-44c5-8cc4-6291d65aad44,Total arrivals,Total International Arrivals,UNWTO,1.1,Inbound Tourism-Arrivals,Inbound Tourism-Arrivals,Thousands
7,d71570c7-cf71-4c69-b252-e3fb62230f30,Middle East,Total Travelers Originating From Middle East,UNWTO,1.1,Inbound Tourism-Regions,Inbound Tourism-Regions,Thousands


## Update Indiactor Ids

In [121]:
def update_indicator_id(df):
    df_temp = df.copy()
    df_temp['S.'] = df_temp['S.'].astype(str)
    merged_df = df_temp.merge(indicator_df[['indicator_id', 'indicator_source_id', 'indicator_source_name']], 
                            left_on=['S.', 'indicator_source_name'], 
                            right_on= ['indicator_source_id','indicator_source_name' ],
                            how='left')
    return merged_df

for i in df_list:
    df = df_list[i]
    df = update_indicator_id(df)
    df_list[i] = df

## Create `country_indicator_value_table` Backup

In [122]:
# Connect to the SQLite3 database
conn = sqlite3.connect('../../datasets/data_discovery_db.sqlite')
cursor = conn.cursor()

# Create a backup table with the same structure as the original
cursor.execute("CREATE TABLE country_indicator_value_table_backup AS SELECT * FROM country_indicator_value_table WHERE 1=0;")

# Copy the data from the original table to the backup table
cursor.execute("INSERT INTO country_indicator_value_table_backup SELECT * FROM country_indicator_value_table;")

# Commit the changes and close the connection
conn.commit()
conn.close()


In [123]:
# def insert_un_indicator_values(df, conn):
#     df_filtered = df[['indicator_id', 'iso3_country_code', 'indicator_year', 'indicator_value']]

#     cursor = conn.cursor()
    
#     # Construct the SQL statement for insertion
#     sql = """
#         INSERT INTO country_indicator_value_table (
#             indicator_id, 
#             iso3_country_code, 
#             indicator_year, 
#             indicator_value 
#         ) 
#         VALUES (?, ?, ?, ?);
#     """
    
#     # Use df.iterrows() to iterate over dataframe rows and execute SQL insert statement for each row
#     for _, row in df_filtered.iterrows():
#         cursor.execute(sql, (
#             row['indicator_id'],
#             row['iso3_country_code'],
#             row['indicator_year'],
#             row['indicator_value']
#         ))
    
#     # Commit the changes
#     conn.commit()


def insert_un_indicator_values(df, conn):
    df_filtered = df[['indicator_id', 'iso3_country_code', 'indicator_year', 'indicator_value']]

    cursor = conn.cursor()
    
    # Construct the SQL statement for insertion with ON CONFLICT clause for SQLite
    sql = """
        INSERT INTO country_indicator_value_table (
            indicator_id, 
            iso3_country_code, 
            indicator_year, 
            indicator_value 
        ) 
        VALUES (?, ?, ?, ?)
        ON CONFLICT(indicator_id, iso3_country_code, indicator_year) DO UPDATE SET
            indicator_value = excluded.indicator_value;
    """
    
    # Use df.iterrows() to iterate over dataframe rows and execute SQL insert statement for each row
    for _, row in df_filtered.iterrows():
        try:
            cursor.execute(sql, (
                row['indicator_id'],
                row['iso3_country_code'],
                row['indicator_year'],
                row['indicator_value']
            ))
        except Exception as e:
            print(f"Failed to insert/update due to error: {e}")
            # Optionally, handle specific errors or rollback/ignore
    
    # Commit the changes
    conn.commit()


## Insert Data into `country_indicator_value_table`

In [124]:
# Connect to the SQLite3 database
conn = sqlite3.connect('../../datasets/data_discovery_db.sqlite')

for i in df_list:
    df = df_list[i]
    insert_un_indicator_values(df, conn)

# Close the connection
conn.close()


## View Changes in `country_indicator_value_table`

In [125]:
# Connect to the database (it will be created if it doesn't exist)
conn = sqlite3.connect('../../datasets/data_discovery_db.sqlite')
cursor = conn.cursor()

# Retrieve data from each table and store in separate DataFrames
#country_df = pd.read_sql_query("SELECT * FROM country_table", conn)
country_indicator_value_df = pd.read_sql_query("SELECT * FROM country_indicator_value_table", conn)
#indicator_df = pd.read_sql_query("SELECT * FROM indicator_table", conn)

# Close the database connection
conn.close()

country_indicator_value_df

,indicator_id,iso3_country_code,indicator_year,indicator_value
0,01fbcfcd-cf29-44c5-8cc4-6291d65aad44,DZA,1995,520.0
1,add77f30-08af-490f-b787-e0a4c86f6dfc,DZA,1995,NaN
2,42cc8507-fcc5-4af4-9a08-d8e5f6b0ca5b,DZA,1995,NaN
3,01fbcfcd-cf29-44c5-8cc4-6291d65aad44,AGO,1995,NaN
4,add77f30-08af-490f-b787-e0a4c86f6dfc,AGO,1995,9.0
...,...,...,...,...
63135,849bdd0a-4d70-45ce-af9d-d326f73523ef,ZWE,2022,NaN
63136,7fcd796e-419c-4b5e-bafd-f2917f4efe7d,ZWE,2022,NaN
63137,defb9f62-d1f3-4901-b0b3-5d940c9c70ab,ZWE,2022,NaN
63138,0c7656f2-59b7-461a-a79b-89aac989cffb,ZWE,2022,NaN


In [126]:
conn = sqlite3.connect('../../datasets/data_discovery_db.sqlite')
cursor = conn.cursor()
cursor.execute("""
UPDATE indicator_table 
SET indicator_ddt_cat = 'Inbound Tourism-Expenditure' 
WHERE indicator_ddt_cat = 'Indbound Tourism- Expenditure';
""")
cursor.execute("""
UPDATE indicator_table 
SET indicator_ddt_cat = 'Inbound Tourism-Arrivals' 
WHERE indicator_ddt_cat = ' Inbound Tourism-Arrivals';
""")
cursor.execute("""
UPDATE indicator_table 
SET indicator_ddt_cat = 'Outbound Tourism-Departures' 
WHERE indicator_ddt_cat = 'Outbound Tourism';
""")
cursor.execute("""
UPDATE indicator_table 
SET indicator_ddt_cat = 'Inbound Tourism-Arrivals' 
WHERE indicator_ddt_cat = 'Inbound Tourism';
""")
indicator_df = pd.read_sql_query("SELECT * FROM indicator_table", conn)

conn.commit()
conn.close()


# Save to Excel

In [158]:
conn = sqlite3.connect('../../datasets/data_discovery_db.sqlite')
cursor = conn.cursor()

# Retrieve data from each table and store in separate DataFrames
country_df = pd.read_sql_query("SELECT * FROM country_table", conn)
country_indicator_value_df = pd.read_sql_query("SELECT * FROM country_indicator_value_table", conn)
indicator_df = pd.read_sql_query("SELECT * FROM indicator_table", conn)

# Close the database connection
conn.close()
df_ddt = country_indicator_value_df
df_ddt = df_ddt.merge(indicator_df[['indicator_id', 'indicator_ddt_name','indicator_source', 'indicator_ddt_cat']], left_on='indicator_id', right_on='indicator_id', how= 'left')
df_ddt = df_ddt.merge(country_df[['iso3_country_code', 'country_name', 'region_name']], left_on='iso3_country_code', right_on='iso3_country_code', how = 'left')
df_ddt.dropna(subset=['indicator_value'], inplace=True)
df_ddt.loc[df_ddt['indicator_ddt_name'] == 'Total Expenditure\xa0 on International Tourism', 'indicator_ddt_name'] = 'Total Expenditure on International Tourism'
df_ddt = df_ddt.merge(indicator_df[['indicator_id', 'units']], on='indicator_id', how='left')
df_ddt = df_ddt.drop(columns=['indicator_id'])

In [159]:
# Assuming df is your DataFrame
# Step 1: Filter for the relevant expenditures
expenditures_df = df_ddt[
    (df_ddt['indicator_ddt_name'].isin(['Expenditure on Passenger Transport', 'Expenditure on Travel'])) &
    (df_ddt['indicator_ddt_cat'] == 'Inbound Tourism-Expenditure')
]

# Step 2: Group by country and year, then sum the expenditures
grouped_expenditures = expenditures_df.groupby(['country_name', 'indicator_year']).agg(
    total_expenditure=('indicator_value', 'sum')
).reset_index()

# Step 3: Prepare new indicator entries
grouped_expenditures['indicator_ddt_name'] = 'Total Expenditure on Inbound Tourism'
grouped_expenditures['indicator_ddt_cat'] = 'Inbound Tourism-Expenditure'
grouped_expenditures['indicator_source'] = 'UNWTO'
grouped_expenditures['region_name'] = grouped_expenditures['country_name'].map(df_ddt.drop_duplicates('country_name').set_index('country_name')['region_name'])
grouped_expenditures['iso3_country_code'] = grouped_expenditures['country_name'].map(df_ddt.drop_duplicates('country_name').set_index('country_name')['iso3_country_code'])
grouped_expenditures['units'] = 'US$ Millions'  # assuming the unit of expenditure is in USD
grouped_expenditures.rename(columns={'total_expenditure' : 'indicator_value'}, inplace = True)
# Step 4: Append the new rows to the original DataFrame
df_ddt = pd.concat([df_ddt, grouped_expenditures], ignore_index=True)


In [160]:
# Assuming df is your DataFrame
# Step 1: Filter for the relevant expenditures
expenditures_df = df_ddt[
    (df_ddt['indicator_ddt_name'].isin(['Expenditure on Passenger Transport', 'Expenditure on Travel'])) &
    (df_ddt['indicator_ddt_cat'] == 'Outbound Tourism-Expenditure')
]

# Step 2: Group by country and year, then sum the expenditures
grouped_expenditures = expenditures_df.groupby(['country_name', 'indicator_year']).agg(
    total_expenditure=('indicator_value', 'sum')
).reset_index()

# Step 3: Prepare new indicator entries
grouped_expenditures['indicator_ddt_name'] = 'Total Expenditure on Outbound Tourism'
grouped_expenditures['indicator_ddt_cat'] = 'Outbound Tourism-Expenditure'
grouped_expenditures['indicator_source'] = 'UNWTO'
grouped_expenditures['region_name'] = grouped_expenditures['country_name'].map(df_ddt.drop_duplicates('country_name').set_index('country_name')['region_name'])
grouped_expenditures['iso3_country_code'] = grouped_expenditures['country_name'].map(df_ddt.drop_duplicates('country_name').set_index('country_name')['iso3_country_code'])
grouped_expenditures['units'] = 'US$ Millions'  # assuming the unit of expenditure is in USD
grouped_expenditures.rename(columns={'total_expenditure' : 'indicator_value'}, inplace = True)
# Step 4: Append the new rows to the original DataFrame
df_ddt = pd.concat([df_ddt, grouped_expenditures], ignore_index=True)


### World Bank

In [37]:
df_ddt = pd.read_excel(r'../../../datasets/ddt/ddt_complete.xlsx', sheet_name='Complete UNWTO')
df_ddt = df_ddt.drop(columns=['Unnamed: 0'])

In [38]:
df_ddt.head()

,iso3_country_code,indicator_year,indicator_value,indicator_ddt_name,indicator_source,indicator_ddt_cat,country_name,region_name,units
0,DZA,1995,520.0,Total International Arrivals,UNWTO,Inbound Tourism-Arrivals,Algeria,Northern Africa,Thousands
1,AGO,1995,9.0,Total International Overnight Visitors (Tourists),UNWTO,Inbound Tourism-Arrivals,Angola,Central Africa,Thousands
2,BEN,1995,580.0,Total International Arrivals,UNWTO,Inbound Tourism-Arrivals,Benin,Western Africa,Thousands
3,BEN,1995,138.0,Total International Overnight Visitors (Tourists),UNWTO,Inbound Tourism-Arrivals,Benin,Western Africa,Thousands
4,BWA,1995,636.0,Total International Arrivals,UNWTO,Inbound Tourism-Arrivals,Botswana,Southern Africa,Thousands


In [23]:
wb_indic = [{'id': 'ST.INT.ARVL', 'value': 'International tourism, number of arrivals', 'units' : 'Units', 'indicator_ddt_name' : 'Total International Arrivals', 'indicator_ddt_cat':  'Inbound Tourism-Arrivals'},
 {'id': 'ST.INT.DPRT', 'value': 'International tourism, number of departures', 'units' : 'Units' , 'indicator_ddt_name' :'Total International Departures', 'indicator_ddt_cat': 'Outbound Tourism-Departures'},
 {'id': 'ST.INT.RCPT.CD',
  'value': 'International tourism, receipts (current US$)', 'units' : 'Current US$', 'indicator_ddt_name' :'Total Expenditure on Inbound Tourism', 'indicator_ddt_cat': 'Inbound Tourism-Expenditure'},
 {'id': 'ST.INT.RCPT.XP.ZS',
  'value': 'International tourism, receipts (% of total exports)', 'units' : '(% of Total Exports)', 'indicator_ddt_name' :'Total Expenditure on Inbound Tourism (Percentage)', 'indicator_ddt_cat': 'Inbound Tourism-Expenditure'},
 {'id': 'ST.INT.TRNR.CD',
  'value': 'International tourism, receipts for passenger transport items (current US$)', 'units' : 'Current US$', 'indicator_ddt_name' :'Expenditure on Passenger Transport', 'indicator_ddt_cat': 'Inbound Tourism-Expenditure'},
 {'id': 'ST.INT.TRNX.CD',
  'value': 'International tourism, expenditures for passenger transport items (current US$)', 'units' : 'Current US$', 'indicator_ddt_name' :'Expenditure on Passenger Transport', 'indicator_ddt_cat': 'Outbound Tourism-Expenditure'},
 {'id': 'ST.INT.TVLR.CD',
  'value': 'International tourism, receipts for travel items (current US$)', 'units' : 'Current US$', 'indicator_ddt_name' :'Expenditure on Travel', 'indicator_ddt_cat': 'Inbound Tourism-Expenditure'},
 {'id': 'ST.INT.TVLX.CD',
  'value': 'International tourism, expenditures for travel items (current US$)', 'units' : 'Current US$', 'indicator_ddt_name' : 'Expenditure on Travel', 'indicator_ddt_cat': 'Outbound Tourism-Expenditure'}, 
 {'id': 'ST.INT.XPND.CD',
  'value': 'International tourism, expenditures (current US$)', 'units' : 'Current US$', 'indicator_ddt_name' :'Total Expenditure on Outbound Tourism', 'indicator_ddt_cat': 'Outbound Tourism-Expenditure'},
 {'id': 'ST.INT.XPND.MP.ZS',
  'value': 'International tourism, expenditures (% of total imports)', 'units' : '(% of Total Imports)', 'indicator_ddt_name' : 'Total Expenditure on Outbound Tourism (Percentage)', 'indicator_ddt_cat': 'Outbound Tourism-Expenditure'}]

In [34]:
wb_df_list = []
for each in wb_indic:
    wb_df = pd.read_excel(r'../../../datasets/world_bank/world_bank_indicators.xlsx', sheet_name= each['id'])

        # Rename the 'economy' column to 'iso3_country_code'
    wb_df.rename(columns={'economy': 'iso3_country_code'}, inplace=True)

    # Melt the DataFrame to convert year columns to rows and strip 'YR' prefix from the year column
    wb_melted = wb_df.melt(id_vars=['iso3_country_code'], var_name='indicator_year', value_name='indicator_value')

    # Strip 'YR' from the 'indicator_year' values to leave only the year
    wb_melted['indicator_year'] = wb_melted['indicator_year'].str.replace('YR', '').astype(int)
    wb_melted = wb_melted.merge(data[['iso3_code', 'sub_region', 'country']], left_on='iso3_country_code', right_on='iso3_code')
    wb_melted.drop(columns=['iso3_code'], inplace=True)
    wb_melted.rename(columns={'country' : 'country_name', 'sub_region': 'region_name'}, inplace=True)
    wb_melted['indicator_ddt_name'] = each['indicator_ddt_name']
    wb_melted['units'] = each['units']
    wb_melted['indicator_ddt_cat'] = each['indicator_ddt_cat']
    wb_melted['indicator_source'] = 'World Bank'

    wb_df_list.append(wb_melted)

wb_concat = pd.concat(wb_df_list, ignore_index=True)
wb_concat.head()

,iso3_country_code,indicator_year,indicator_value,region_name,country_name,indicator_ddt_name,units,indicator_ddt_cat,indicator_source
0,AGO,1995,9000.0,Central Africa,Angola,Total International Arrivals,Units,Inbound Tourism-Arrivals,World Bank
1,AGO,1996,21000.0,Central Africa,Angola,Total International Arrivals,Units,Inbound Tourism-Arrivals,World Bank
2,AGO,1997,45000.0,Central Africa,Angola,Total International Arrivals,Units,Inbound Tourism-Arrivals,World Bank
3,AGO,1998,52000.0,Central Africa,Angola,Total International Arrivals,Units,Inbound Tourism-Arrivals,World Bank
4,AGO,1999,45000.0,Central Africa,Angola,Total International Arrivals,Units,Inbound Tourism-Arrivals,World Bank


In [39]:
df_ddt = pd.concat([df_ddt, wb_concat], ignore_index=True)

## To Excel

In [43]:
df_ddt.to_excel(r'../../../datasets/ddt/ddt_complete.xlsx', sheet_name='Complete DDT')